Criação das Instâncias de serviços

In [ ]:
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions
from awsglue.transforms import Map, DropFields, ApplyMapping

sc = SparkContext()
context = GlueContext(sc)

In [ ]:
# s3://network.cubo.datalake/airtable/raw/prod/2023-01-02-20-16-19/data/Selo Cubo Startups/Corporates Ativas/Grid view/
path = "s3://network.cubo.datalake/airtable/raw/prod/"
path = path + "2023-01-02-20-16-19/data/"
path = path + "Selo Cubo Startups/Parceiros Ativos/Grid view/"
print(path)
dataframe = context.create_dynamic_frame.from_options(
                connection_type='s3',
                connection_options={
                    'paths': [path],
                    'recurse': True
                },
                format='json'
            )
dataframe.count()

In [ ]:
dataframe.printSchema()

In [ ]:
mapping = []
for item in dataframe.unnest().toDF().dtypes:
    if item[0].split('.')[0] == "fields":
        if len(item[0].split(".")) == 2:
            mapping.append((item[0], item[0].split('.')[1]))
    else:
        mapping.append((item[0], item[0].split('.')[0]))

In [ ]:
newDf = dataframe.apply_mapping(mapping).resolveChoice(specs=[
    ("Faturamento 2022", "cast:decimal"),
    ("Valor a captar / Faturamento previsto", "cast:decimal"),
    ("Previsão do faturamento 2023", "cast:decimal"),
    ("Captable", "cast:decimal"),
    ("Faturamento 2021", "cast:decimal"),
    ("Turnover", "cast:decimal"),
    ("Δ Fat 21-22", "cast:decimal"),
    ("Δ Clientes 21-22", "cast:decimal"),
    ("Δ Fat 22-23", "cast:decimal"),
    ("Δ Fat 21-23", "cast:decimal"),
    ("Δ Fat 21-22/22-23", "cast:decimal"),
    ("Δ Clientes 22-23", "cast:decimal"),
    ("Δ Clientes 21-23", "cast:decimal"),
    ("Δ Clientes 21-22/22-23", "cast:decimal"),
    ("Δ Time 22-23", "cast:decimal"),
    ("Δ Time 21-22/22-23", "cast:decimal"),
    ("PERFORMANCE", "cast:decimal"),
    ("CRESCIMENTO", "cast:decimal"),
    ("INVESTIMENTO", "cast:decimal"),
    ("PESSOAS", "cast:decimal"),
    ("Δ Time 21-23", "cast:decimal"),
    ("Δ Time 21-22", "cast:decimal"),
    ("Δ Time 21-22/22-23", "cast:decimal")
])
newDf.printSchema()


In [ ]:
newDf.select_fields("CRESCIMENTO").toDF().show()

In [ ]:
# path = "s3://network.cubo.datalake/airtable/proc/prod/"
# path = path + "2022-12-29-15-21-18/data/"
# path = path + "Selo Cubo Startups/Selo Cubo 2023/📝 Respostas Selo Cubo/"
# context.write_dynamic_frame.from_options(
#             frame=newDf,
#             connection_type="s3",
#             format="parquet",
#             connection_options={
#                 "path": path,
#                 "partitionKeys": [],
#             },
#             format_options={"compression": "gzip"},
#             transformation_ctx="S3Destination",
#         )